In [1]:
from ase.io import read
from lammps import lammps
import os

os.environ["LAMMPS_COMMAND"] = "/home/dux/lammps/src/lmp_serial"
os.environ["LAMMPS_POTENTIALS"] = "/home/dux/lammps/potentials/"

main_dir = "/home/dux/surface_sampling/sgmc_surf/GaN"

# define necessary file locations
lammps_data_file = f"{main_dir}/lammps.data"
lammps_in_file = f"{main_dir}/lammps.in"
potential_file = f"GaN.tersoff"
atoms = ["Ga", "N"]
lammps_out_file = f"{main_dir}/lammps.out"
cif_from_lammps_file = f"{main_dir}/lammps.cif"

slab = read("/home/dux/surface_sampling/sgmc_surf/GaN/GaN_0001_3x3_pymatgen/runs500_temp1_adsatoms12_alpha0.995_20220518-172644/optim_slab_run_500.cif")

In [2]:
# write current surface into lammps.data
slab.write(
    lammps_data_file, format="lammps-data", units="real", atom_style="atomic"
)
TEMPLATE = """
clear
atom_style atomic
units metal
boundary p p p
atom_modify sort 0 0.0

# read_data /path/to/data.data
read_data {}

### set bulk
group bulk id <= 36
group surface id > 36

### interactions
pair_style tersoff
# pair_coeff * * /path/to/potential Atom1 Atom2
pair_coeff * * {} {} {}
mass 1 69.723000
mass 2 14.007000

### run
reset_timestep 0
velocity surface create 100.0 87287 dist gaussian

fix 1 all nvt temp 100.0 0.1 $(100.0*dt)

# set bulk forces and velocities to 0
fix 2 bulk setforce 0.0 0.0 0.0
velocity bulk set 0.0 0.0 0.0

thermo 10 # output thermodynamic variables every N timesteps

thermo_style custom step temp pe press cpu pxx pyy pzz pxy pxz pyz ke etotal vol lx ly lz atoms
thermo_modify flush yes format float %23.16g
# min_style cg
# minimize 1e-5 1e-5 500 10000

timestep 0.001
run 3000

# write_data /path/to/data.out
write_data {}
print "_end of MD_"
log /dev/stdout

"""

# write lammps.in file
with open(lammps_in_file, "w") as f:
    f.writelines(
        TEMPLATE.format(lammps_data_file, potential_file, *atoms, lammps_out_file)
    )

lmp = lammps()
# print("LAMMPS Version: ", lmp.version())

# run the LAMMPS here
lmp.file(lammps_in_file)
lmp.close()

# Read from LAMMPS out
opt_slab = read(lammps_out_file, format="lammps-data", style="atomic")

atomic_numbers_dict = {1: 31, 2: 7}  # 1: Ga, 2: N
actual_atomic_numbers = [
    atomic_numbers_dict[x] for x in opt_slab.get_atomic_numbers()
]
print(f"actual atomic numbers {actual_atomic_numbers}")
# correct_lammps = new_slab.copy()
opt_slab.set_atomic_numbers(actual_atomic_numbers)
opt_slab.calc = slab.calc
opt_slab.write(cif_from_lammps_file)

OSError: libfftw3_omp.so.3: cannot open shared object file: No such file or directory

In [ ]:
opt_slab.get_positions()

In [ ]:
slab.get_positions()